### Installing libraries

In [1]:
# Fixed Cell 1
# We install the new libraries but DO NOT force upgrade torch/torchvision
# to avoid breaking the Kaggle environment.
!pip install -q -U transformers peft datasets bitsandbytes trl accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB

### Conifuring secrets for huggingface

In [2]:
import torch
import os
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTConfig, SFTTrainer

# Disable WandB to prevent connection/protobuf errors
os.environ["WANDB_DISABLED"] = "true"

2025-12-16 06:12:24.686639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765865544.871902      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765865544.928064      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

### Weights configuration:

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

### Loading pretrained model from kaggle

In [4]:
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Disable caching for training and force config consistency
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

### again loading, this time correctly

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. Define Model Name
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

# 2. Configure Quantization (Cell 3)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# 3. Load Model (Cell 4)
print("Loading Model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

# 4. Load Tokenizer (Cell 5 - The missing piece!)
print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Done! Now you can run Cell 6 (Dataset) and Cell 7/8 (Training).")

Loading Model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Tokenizer...
Done! Now you can run Cell 6 (Dataset) and Cell 7/8 (Training).


### Loading dataset in urdu

In [16]:
from datasets import load_dataset

# 1. Load the working Urdu Alpaca dataset
print("Downloading real Urdu dataset...")
dataset = load_dataset("mahwizzzz/urdu_alpaca_yc_filtered", split="train")

# 2. Inspect the raw data to see column names (Just to be sure!)
print("Raw columns:", dataset.column_names)

# 3. Format function (UPDATED to match the actual column names)
def format_instruction(sample):
    # The columns are named 'urdu_instruction', 'urdu_input', 'urdu_output'
    instruction = sample['urdu_instruction']
    context = sample['urdu_input']
    response = sample['urdu_output']
    
    # If there is extra context (input), add it to the user prompt
    if context and len(str(context)) > 1:
        user_prompt = f"{instruction}\nInput: {context}"
    else:
        user_prompt = instruction
        
    # Create the final text string for the model
    full_text = f"User: {user_prompt}\nAssistant: {response}"
    return {"text": full_text}

# 4. Apply formatting to the whole dataset
print("Formatting dataset...")
dataset = dataset.map(format_instruction)

# 5. Filter and Select
# We select 500 examples.
dataset = dataset.shuffle(seed=42).select(range(500))

print(f"\nFINAL DATASET READY: {len(dataset)} examples.")
print("-" * 50)
print(dataset[0]['text'])
print("-" * 50)

Raw columns: ['urdu_instruction', 'urdu_input', 'urdu_output']
Formatting dataset...

FINAL DATASET READY: 500 examples.
--------------------------------------------------
User: ایس کیو ایل ڈیٹا بیس کے لئے ایک سوال لکھیں۔
Input: ایک مووی ڈیٹا بیس جس میں 'فلموں' کی ایک ٹیبل اور 'اداکاروں' کی ایک میز ہوتی ہے۔
Assistant: فلموں کا عنوان منتخب کریں، actors.name
فلموں سے
اندرونی طور پر اداکاروں میں شامل ہو جائیں 
او این movies.actor_id = actors.actor_id
فلموں کے عنوان کے مطابق آرڈر؛
--------------------------------------------------


### Preparing model for training, using PEFT

In [17]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = get_peft_model(model, peft_config)

# *** STABILITY FIX ***
# We force the trainable LoRA adapters to Float32. 
# This ensures gradients are calculated in a format the T4 GPU supports.
for name, param in model.named_parameters():
    if "lora" in name:
        param.data = param.data.to(torch.float32)

model.print_trainable_parameters()

trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


### Training model on 1 epoch

In [18]:
sft_config = SFTConfig(
    output_dir="./results",
    dataset_text_field="text",
    max_length=256,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    logging_steps=10,
    
    # *** THE KEY SETTING ***
    # strictly False to prevent the "unscale_cuda" crash
    fp16=False,
    bf16=False,
    
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
)

print("Starting training...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Starting training...


Step,Training Loss
10,1.819200
20,1.626000
30,1.649200
40,1.513100
50,1.524400
60,1.468100


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=63, training_loss=1.598415177965921, metrics={'train_runtime': 1504.2836, 'train_samples_per_second': 0.332, 'train_steps_per_second': 0.042, 'total_flos': 2014408115945472.0, 'train_loss': 1.598415177965921, 'entropy': 1.5641355514526367, 'num_tokens': 116787.0, 'mean_token_accuracy': 0.6126447518666586, 'epoch': 1.0})

### Testing it out

In [19]:
def generate_response(prompt):
    # Format the prompt using the chat template
    messages = [
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenize and move to GPU
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,      # Limit the response length
        temperature=0.7,         # Creativity level
        top_k=50,
        top_p=0.9,
        do_sample=True,          # Enable sampling for variety
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the assistant's reply (stripping the prompt)
    # The split ensures we don't print the user's question twice
    response = decoded_output.split("assistant\n")[-1].strip()
    return response

# --- Run a Test ---
test_prompt = "اور سناو، تمہارا نام کیا ہے؟"
print(f"User: {test_prompt}")
print("-" * 50)
print(f"Assistant: {generate_response(test_prompt)}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_values=None`.


User: اور سناو، تمہارا نام کیا ہے؟
--------------------------------------------------


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Assistant: م:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


### Saving model:

In [20]:
import shutil

# 1. Save the model (Adapters) and Tokenizer locally
new_model_name = "Qwen2.5-Urdu-Finetune"
print(f"Saving model to {new_model_name}...")

model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

print("Model saved successfully!")

# 2. (Optional) Zip the folder so you can download it easily
print("Zipping folder for download...")
shutil.make_archive(new_model_name, 'zip', new_model_name)

print(f"Zip file created: {new_model_name}.zip")

# 3. Trigger Download (For Google Colab users)
# If you are on Kaggle, the file will appear in the 'Output' section instead.
try:
    files.download(f"{new_model_name}.zip")
except:
    print("Auto-download failed. Please download the ZIP file manually from the file explorer.")

Saving model to Qwen2.5-Urdu-Finetune...
Model saved successfully!
Zipping folder for download...
Zip file created: Qwen2.5-Urdu-Finetune.zip
Auto-download failed. Please download the ZIP file manually from the file explorer.


### For using it afterwards:

In [ ]:
# Example of how to load it back later
from peft import PeftModel

# 1. Load Base
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct", ...)

# 2. Load YOUR Adapters
model = PeftModel.from_pretrained(base_model, "Qwen2.5-Urdu-Finetune")